In [ ]:
# import hashlib
# import json
# import random
# from hashlib import md5

# import requests


# class BaiDuFanyi:
#     def __init__(self, appKey, appSecret):
#         self.url = "https://fanyi-api.baidu.com/api/trans/vip/translate"
#         self.appid = appKey
#         self.secretKey = appSecret
#         self.fromLang = "auto"
#         self.toLang = "en"
#         self.salt = random.randint(32768, 65536)
#         self.header = {"Content-Type": "application/x-www-form-urlencoded"}

#     def BdTrans(self, text):
#         sign = self.appid + text + str(self.salt) + self.secretKey
#         md = hashlib.md5()
#         md.update(sign.encode(encoding="utf-8"))
#         sign = md.hexdigest()
#         data = {
#             "appid": self.appid,
#             "q": text,
#             "from": self.fromLang,
#             "to": self.toLang,
#             "salt": self.salt,
#             "sign": sign,
#         }
#         response = requests.post(self.url, params=data, headers=self.header)  # 发送post请求
#         text = response.json()  # 返回的为json格式用json接收数据
#         # print(text)
#         try:
#             results = text["trans_result"][0]["dst"]
#         except Exception:
#             results = ""
#         return results


# if __name__ == "__main__":
#     appKey = "20221012001387816"  # 你在第一步申请的APP ID
#     appSecret = "ThXoAK3TTPMmnaOKX0yF"  # 公钥
#     BaiduTranslate_test = BaiDuFanyi(appKey, appSecret)
#     Results = BaiduTranslate_test.BdTrans("Hello, World!")  # 要翻译的词组
#     print(Results)

In [ ]:
import json

import pandas as pd
import pyrootutils

root = pyrootutils.setup_root(".")

In [ ]:
data_file_list = [
    "test_nonrumor_format.json",
    "test_rumor_format.json",
    "train_nonrumor_format.json",
    "train_rumor_format.json",
]

In [ ]:
dfs = [pd.read_json(root / "data" / "MM17-WeiboRumorSet" / f, lines=True) for f in data_file_list]

In [ ]:
all_data = pd.concat(dfs, ignore_index=True)

In [ ]:
all_data.shape

In [ ]:
all_data["title"] = all_data["title"].apply(lambda x: x[0] if isinstance(x, list) else x)

In [ ]:
all_data.head(3)

In [ ]:
import jionlp
from tqdm import tqdm

In [ ]:
def clean_text(text: str):
    text = jionlp.remove_html_tag(text)
    text = jionlp.remove_exception_char(text)
    text = jionlp.remove_redundant_char(text)
    text = jionlp.replace_url(text)
    text = jionlp.replace_id_card(text)
    text = jionlp.replace_phone_number(text)
    text = jionlp.replace_email(text)
    text = jionlp.replace_ip_address(text)
    text = jionlp.replace_qq(text)
    return text

In [ ]:
all_data["cleaned_text"] = all_data["title"].apply(clean_text)

In [ ]:
all_data[["title", "cleaned_text"]].sample(3)

In [ ]:
# all_data["cleaned_text"] = all_data["title"].apply(jionlp.clean_text)

In [ ]:
all_data[["title", "cleaned_text"]].sample(3)

In [ ]:
# translator = BaiDuFanyi(appKey, appSecret)

In [ ]:
# tqdm.pandas(desc="Translating")

# all_data["translated_text"] = all_data.cleaned_text.progress_apply(translator.BdTrans)

# Tencent translate api

In [ ]:
from tencentcloud.common import credential
from tencentcloud.common.exception.tencent_cloud_sdk_exception import (
    TencentCloudSDKException,
)
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.tmt.v20180321 import models, tmt_client


def tencent_translate(text: str, source: str = "auto", target: str = "en"):
    try:
        # 实例化一个认证对象，入参需要传入腾讯云账户secretId，secretKey,此处还需注意密钥对的保密
        # 密钥可前往https://console.cloud.tencent.com/cam/capi网站进行获取
        cred = credential.Credential(
            "AKIDGQNCrLchZcR2SfdO0SgV9csvnTxDJH5f", "67ybtBSXNzPdfTAh0DG6iW5TdYQ4Zr8M"
        )
        # 实例化一个http选项，可选的，没有特殊需求可以跳过
        httpProfile = HttpProfile()
        httpProfile.endpoint = "tmt.tencentcloudapi.com"

        # 实例化一个client选项，可选的，没有特殊需求可以跳过
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        # 实例化要请求产品的client对象,clientProfile是可选的
        client = tmt_client.TmtClient(cred, "ap-guangzhou", clientProfile)

        # 实例化一个请求对象,每个接口都会对应一个request对象
        req = models.TextTranslateRequest()
        params = {
            "Action": "TextTranslate",
            "Version": "2018-03-21",
            "SourceText": text,
            "Source": source,
            "Target": target,
            "ProjectId": 0,
        }
        req.from_json_string(json.dumps(params))

        # 返回的resp是一个TextTranslateResponse的实例，与请求对象对应
        resp = client.TextTranslate(req)
        # 输出json格式的字符串回包
        return json.loads(resp.to_json_string())["TargetText"]

    except TencentCloudSDKException as err:
        tqdm.write(str(err))
        return ""

In [ ]:
tqdm.pandas(desc="Translating")
all_data["translated_text"] = all_data.cleaned_text.progress_apply(tencent_translate)

In [ ]:
all_data[["title", "cleaned_text", "translated_text"]].sample(3)

In [ ]:
all_data["id"] = all_data.meta_data.apply(lambda x: str(x[0]))

In [ ]:
all_data.translated_text.apply(lambda x: len(x.split())).describe()

In [ ]:
test_data = pd.concat(dfs[:2], ignore_index=True)
train_data = pd.concat(dfs[2:], ignore_index=True)

In [ ]:
train_data["id"] = train_data.meta_data.apply(lambda x: str(x[0]))
test_data["id"] = test_data.meta_data.apply(lambda x: str(x[0]))

In [ ]:
train_data["translated_text"] = train_data.id.apply(
    lambda x: all_data[all_data.id == x].translated_text.values[0]
)

In [ ]:
test_data["translated_text"] = test_data.id.apply(
    lambda x: all_data[all_data.id == x].translated_text.values[0]
)

In [ ]:
train_data[["title", "translated_text"]].sample(3)

In [ ]:
train_data.to_json(
    root / "data" / "MM17-WeiboRumorSet" / "train_data_tencent_translated_cleaned.json",
    orient="records",
    lines=True,
    force_ascii=False,
)
test_data.to_json(
    root / "data" / "MM17-WeiboRumorSet" / "test_data_tencent_translated_cleaned.json",
    orient="records",
    lines=True,
    force_ascii=False,
)

In [ ]:
train_data.translated_text

In [ ]:
train_data.title.sample(3)